In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
import random
import pandas as pd
from matplotlib import gridspec
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import ttest_ind, normaltest
from sklearn.svm import SVC, SVR
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_selection import RFE, SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.manifold import TSNE

In [2]:
def FE_cv(scaler, selector, dfX, dfy, method, fe_niter=30):
    '''
    scaler (e.g. StandardScaler) and selector (e.g. RFE) are scikit-learn objects
    dfX: feature matrix
    dfy: response vector
    method: 1: LOO; 2: 80/20
    fe_niter: number of feature elimination iterations
    
    returns frequency (max. is fe_niter) of each feature column in terms of 
    the number of times selcted by the selector, in descending order
    '''
    X = np.array(dfX)
    ylab = np.array(dfy)
    pipeline = Pipeline(steps=[('scaler', scaler),('selector', selector)])

    selected = []
    
    if method==1:  # leave one out
        N = X.shape[0]
        acc = 0
        for i in range(N):
            mask = np.ones((N,), dtype=bool)
            mask[i] = 0
            # scaler based on N-1 cases
            Xtrain = X[mask]
            pipeline.fit(Xtrain, ylab[mask])
            selected.append(np.nonzero(selector.get_support())[0])
    else:  # randomly select 80% as training set
        for k in range(fe_niter):
            trainX, testX, trainy, testy = train_test_split(X, ylab, test_size=0.2, random_state=k)
            pipeline.fit(trainX, trainy)
            selected.append(np.nonzero(selector.get_support())[0])
    freq = np.bincount(np.array(selected).flatten())
    columns = list(dfX.columns[np.argsort(freq)[-7:]])
    freq = list(np.sort(freq)[-7:])
    freq.reverse()
    columns.reverse()
    return freq, columns


def cv(Xs, models, scaler, ylab, test_indices, print_loo=False):
    '''
    Xs and models are lists.
    Test using each list of features and each model
    Scaling is applied before inputting into each model
    
    If test_indices is not [], then also test the specific test set
    (using the other for training)
    Otherwise, use the two default validation methods (LOO and 80/20)
    
    Set print_loo to true to print the result for every single case 
    in the LOO validation
    '''
    # number of validation methods
    if test_indices != []:
        n_methods = 3
    else:
        n_methods = 2
        
    results = np.zeros((len(models), n_methods*len(Xs)))
    cv_niter=100  # repeat
    for i in range(len(models)):
        model = models[i]
        pipeline = Pipeline(steps=[('scaler', scaler),('model', model)])
        for j in range(len(Xs)):
            X = Xs[j]
            N = X.shape[0]
            # leave-one-out
            LOO_acc = np.zeros((N,))
            for k in range(N):
                mask = np.ones((N,), dtype=bool)
                mask[k] = 0
                # scaler based on N-1 cases
                Xsub = X[mask]
                # test case
                Xtest = np.array(X.iloc[k,:]).reshape(1, -1)

                pipeline.fit(Xsub, ylab[mask])
                LOO_acc[k] = np.mean(pipeline.predict(Xtest) == ylab[k])
            if print_loo: 
                print(np.nonzero(LOO_acc==0)[0])
            results[i, j*n_methods] = np.mean(LOO_acc)
            # 80-20 split
            cv82_acc = np.zeros((cv_niter,))
            for k in range(cv_niter):
                trainX, testX, trainy, testy = train_test_split(X, ylab, test_size=0.2, random_state=k)
                pipeline.fit(trainX, trainy)
                pred = pipeline.predict(testX)
                cv82_acc[k] = np.mean(pred == testy)
            results[i, j*n_methods+1] = np.mean(cv82_acc)
            
            # train test
            if test_indices!=[]:
                trainX = X.drop(test_indices, axis=0)
                trainy = ylab.drop(test_indices, axis=0)
                testX = X.iloc[test_indices,:]
                testy = ylab.iloc[test_indices]
                pipeline = Pipeline(steps=[('scaler', scaler),('model', model)])
                pipeline.fit(trainX, trainy)
                correct = pipeline.predict(testX)==testy
                results[i, j*n_methods+2] = np.mean(correct)
    return results


def remove_highcc(X, cc_cutoff):
    '''
    Remove highly correlated features as defined by Pearson correlation 
    exceeding the cc_cutoff threshold (in terms of magnitude)
    '''
    highc_row, highc_col = np.nonzero(np.abs(np.corrcoef(X.transpose()))>=cc_cutoff) 
    diff_var = highc_row != highc_col
    highc_row, highc_col = highc_row[diff_var], highc_col[diff_var]  
    highc_pair = []; highc_delete = []
    for r,c in zip(highc_row, highc_col):
        if (min(r,c), max(r,c)) not in highc_pair:
            if r<=c:
                highc_delete.append(c)
            else:
                highc_delete.append(r)
            highc_pair.append((min(r,c), max(r,c)))
    mask = np.ones((X.shape[1],), dtype=bool)
    mask[highc_delete] = 0
    return X.iloc[:, mask]

In [3]:
def feature_selection(
           path='AllNewFeatures1.csv', 
           ttest_crit=0.1, 
           cc_cutoff=0.99, 
           fs_method=1,
           fs_niter=30, n_var=7, 
           fs_models=[(SelectKBest(score_func=f_classif,k=5),2), 
                      (RFE(SVC(kernel='linear'), n_features_to_select=5),2), 
                      (RFE(LogisticRegression(solver='liblinear'), n_features_to_select=5),2),
                      (RFE(RandomForestClassifier(n_estimators=20), n_features_to_select=5),2)],
           fs_model_names=['SelectKBest', 'RFE(SVM)', 'RFE(LR)', 'RFE(RF)'],
           scaler=StandardScaler(),
           print_loo=False):
    '''
    path: data file name
    ttest_crit: integer for top N variables; float for cutoff of p-value
    fs_method: 1: 80/20 x 30 rep; 2: LOO; 3: 80% LOO
    n_var: number of variables to show from automated fs
    
    the default values are the ones used in the final results
    '''
    # original data
    allft = pd.read_csv(path)
    ally = allft['Group']
    allX = allft.iloc[:,2:]
    allX = remove_highcc(allX, cc_cutoff)
    # prepare train/test sets
    if fs_method==3:
        trainX, testX, trainy, testy = train_test_split(allX, ally, test_size=7/37, random_state=5)
        print('cases excluded from feature selection:')
        print(np.array(testy.index))
        print(np.array(testy))
    else:
        eval_method = 1
        trainX, trainy = allX.copy(), ally.copy()
    
    ### step 1: t-test on dpX --> take top (n_ttest) : "ttestX"
    np.random.seed(12345678)
    tstat, pval = ttest_ind(trainX[trainy==1],trainX[trainy==0], equal_var=False)
    if ttest_crit < 1:
        n_ttest = sum(pval<=ttest_crit)
    else:
        n_ttest = ttest_crit
    ttestX = trainX.iloc[:,np.argsort(pval)[:n_ttest]]
    print('ttestX.shape:', ttestX.shape)
    
    ### step 3: automatic feature selection
    # to be used for pandas dataframe
    fs_res = [] 
    fs_labels = [(mlab, collab) for mlab in fs_model_names for collab in ['Xs', 'freqs']]
    fs_labels = pd.MultiIndex.from_tuples(fs_labels)
    # union set of frequent variable names
    fs_union_xs = set() 
    if fs_method!=1:
        fs_niter=trainX.shape[0]
    fs_crit = fs_niter * 2/3
    for fs_model, data_input in fs_models:
        if data_input == 1:
            data_input = trainX
        elif data_input == 2:
            data_input = ttestX
        fs, xs = FE_cv(scaler, fs_model, data_input, trainy, 1+(fs_method==1), fs_niter)
        fs_res.append(xs)
        fs_res.append(fs)
        fs_union_xs = fs_union_xs.union(set(np.array(xs)[(np.array(fs)>=fs_crit)]))
    fs_res = np.array(fs_res).transpose()
    fs_table = pd.DataFrame(fs_res, columns=fs_labels)
    print('results from feature selection:')
    print(fs_table) 
    
    return fs_table

In [4]:
def modelling(
           path='AllNewFeatures1.csv', 
           eval_method=1,
           eval_models=[SVC(kernel='linear'), SVC(kernel='rbf'),
                        LogisticRegression(solver='liblinear'), 
                        KNeighborsClassifier(3, algorithm='auto'),
                        KNeighborsClassifier(5, algorithm='auto'),
                        DecisionTreeClassifier(max_depth=2, random_state=0),
                        DecisionTreeClassifier(max_depth=3, random_state=0)],
           eval_model_names=['SVM linear', 'SVM rbf', 'LR', 'KNN 3', 'KNN 5', 'DTree 2', 'DTree 3'],
           customized_Xs=[],
           scaler=StandardScaler(),
           print_loo=False):
    '''
    path: data file name
    ttest_crit: integer for top N variables; float for cutoff of p-value
    eval_method: 1: LOO + 80/20 x 100; 2: LOO + 80/20 x 100 + 80/20 x 1
    
    the default values are the ones used in the final results
    '''
    # original data
    allft = pd.read_csv(path)
    ally = allft['Group']
    allX = allft.iloc[:,2:]
    
    Xs = [] + \
         [allX[cols] for cols in customized_Xs]
    if eval_method==1:
        cv_res = cv(Xs, eval_models, scaler, ally, [], print_loo=print_loo)
        method_labels = ['LOO', '80/20']
    else:
        trainX, testX, trainy, testy = train_test_split(allX, ally, test_size=7/37, random_state=5)
        cv_res = cv(Xs, eval_models, scaler, ally, testy.index, print_loo=print_loo)
        method_labels = ['LOO', '80/20', 'testset']
    Xs_labels = [] + ['X'+str(i+1) for i in range(len(customized_Xs))]
    for x, lab in zip(Xs, Xs_labels):
        print(lab, '   ', list(x.columns))
    
    eval_labels = [(xlab, mlab) for xlab in Xs_labels for mlab in method_labels]
    eval_labels = pd.MultiIndex.from_tuples(eval_labels)
    eval_table = pd.DataFrame(cv_res, columns=eval_labels, index=eval_model_names)    
    print('evaluation results:')
    print(round(eval_table,3)*100)
    return Xs, eval_table

In [5]:
fs_table = feature_selection()

ttestX.shape: (37, 31)
results from feature selection:
               SelectKBest                       RFE(SVM)        \
                        Xs freqs                       Xs freqs   
0        Curvature_mean_PD    26        Curvature_mean_PD    23   
1       CrossArea_mean_PD2    18  Curvature_integration_P    22   
2  Curvature_integration_P    13                 LineDist    18   
3        CrossArea_max_PD2    11        Diameter_normalD2    18   
4         Curvature_mean_P     9      Eccentricity_std_P2    17   
5         Ratio_Dpnordnor1     9                PointDist     6   
6       MajorAxis_mean_PD2     8        MinorAxis_mean_D2     6   

                   RFE(LR)                          RFE(RF)        
                        Xs freqs                         Xs freqs  
0        Curvature_mean_PD    28          Diameter_normalD2    25  
1      Eccentricity_std_P2    22          Curvature_mean_PD    19  
2  Curvature_integration_P    20        Eccentricity_std_P2    15  
3

In [6]:
Xs, cv_table = modelling(
       customized_Xs=[['Curvature_mean_PD','Diameter_normalD2','Eccentricity_std_P2','Curvature_integration_P','CrossArea_mean_PD2'],
                      ['Curvature_mean_PD','Diameter_normalD2','Eccentricity_std_P2','Curvature_integration_P'],
                      ['Diameter_normalD2','Curvature_mean_PD','Eccentricity_std_P2'],
                      ['Diameter_normalD2','Curvature_mean_PD'],
                      ['Diameter_normalD2','Eccentricity_std_P2'],
                      ['Curvature_mean_PD','Eccentricity_std_P2']])

X1     ['Curvature_mean_PD', 'Diameter_normalD2', 'Eccentricity_std_P2', 'Curvature_integration_P', 'CrossArea_mean_PD2']
X2     ['Curvature_mean_PD', 'Diameter_normalD2', 'Eccentricity_std_P2', 'Curvature_integration_P']
X3     ['Diameter_normalD2', 'Curvature_mean_PD', 'Eccentricity_std_P2']
X4     ['Diameter_normalD2', 'Curvature_mean_PD']
X5     ['Diameter_normalD2', 'Eccentricity_std_P2']
X6     ['Curvature_mean_PD', 'Eccentricity_std_P2']
evaluation results:
              X1          X2          X3          X4          X5          X6  \
             LOO 80/20   LOO 80/20   LOO 80/20   LOO 80/20   LOO 80/20   LOO   
SVM linear  94.6  89.4  86.5  88.0  86.5  83.6  83.8  81.8  78.4  72.2  81.1   
SVM rbf     83.8  81.6  86.5  83.4  81.1  77.5  78.4  72.5  75.7  76.8  78.4   
LR          94.6  91.5  86.5  87.0  86.5  82.6  83.8  81.8  78.4  75.6  78.4   
KNN 3       83.8  83.5  83.8  78.1  75.7  72.4  73.0  74.5  64.9  68.6  67.6   
KNN 5       86.5  83.0  83.8  82.1  75.7  75.8  75.

In [8]:
fs_table

SelectKBest                       RFE(SVM)        \
                        Xs freqs                       Xs freqs   
0        Curvature_mean_PD    26        Curvature_mean_PD    23   
1       CrossArea_mean_PD2    18  Curvature_integration_P    22   
2  Curvature_integration_P    13                 LineDist    18   
3        CrossArea_max_PD2    11        Diameter_normalD2    18   
4         Curvature_mean_P     9      Eccentricity_std_P2    17   
5         Ratio_Dpnordnor1     9                PointDist     6   
6       MajorAxis_mean_PD2     8        MinorAxis_mean_D2     6   

                   RFE(LR)                          RFE(RF)        
                        Xs freqs                         Xs freqs  
0        Curvature_mean_PD    28          Diameter_normalD2    25  
1      Eccentricity_std_P2    22          Curvature_mean_PD    19  
2  Curvature_integration_P    20        Eccentricity_std_P2    15  
3                 LineDist    19  Eccentricity_variation_D2    12  
4         Ratio_Dpnordnor1    17           Ratio_Dpnordnor1    10  
5        Diameter_normalD2     8    Curvature_integration_P     9  
6         Ratio_Dpnordnor2     7          MajorAxis_mean_D2     9

In [9]:
subtable = []
names = []
multidx = fs_table.columns
for i in range(len(multidx)):
    if i%2 == 1:
        continue
    idx1, idx2 = multidx[i]
    names.append(idx1)
    subtable.append(fs_table[idx1].set_index(idx2))

In [10]:
alltable = subtable[0]
for i in range(1, len(subtable)):
    alltable = alltable.merge(subtable[i], 'outer', left_index=True, right_index=True, sort=False)
alltable.columns = names
alltable = alltable.fillna(0).astype(int)
alltable = alltable.sort_values('SelectKBest', ascending=True)

In [14]:
alltable

,SelectKBest,RFE(SVM),RFE(LR),RFE(RF)
Xs,,,,
Diameter_normalD2,0,18,8,25
Eccentricity_std_P2,0,17,22,15
Eccentricity_variation_D2,0,0,0,12
LineDist,0,18,19,0
MajorAxis_mean_D2,0,0,0,9
MinorAxis_mean_D2,0,6,0,0
PointDist,0,6,0,0
Ratio_Dpnordnor2,0,0,7,0
MajorAxis_mean_PD2,8,0,0,0
